# Converting experimental data to simulation data model

In [58]:
import pandas as pd
import numpy as np

## Data cleaning

In [59]:
pd.options.mode.chained_assignment = None

def remove_space(x):
    return x.replace(" ","")

def cleaning(df:pd.DataFrame):
    """Return a cleaned dataframe"""
    # Cleaning the columns name : deleting the spaces
    df.columns = df.columns.str.replace(" ","")

    # Keeping important features
    features = ["cname","frame","time","cellno","schnitzno","birth","lifespan","length","area","angle","grate","width","x","y"]
    df = df[features]
    
    # Removing the spaces in cname
    df["cname"] = df["cname"].apply(remove_space)

    return df

def cleaning2(df):
    """Remove the problematic data"""
    # Removing the rows with cname = 0 0T 0H
    df = df.loc[df["cname"]!="0"]
    df = df.loc[df["cname"]!="0T" ]
    df = df.loc[df["cname"]!="0H"]

    # Calculating the max frame and removing the corresponding rows
    # max = df["frame"].max()
    # df = df.loc[df["frame"]!=max]

    return df



## Data reading

In [60]:
df = pd.read_csv("./Data_Durvernoy/coli/141003_MG1655_LB1-fld1.txt",sep="\t")
file_name = "Coli7"
# df = pd.read_csv("./Data_Durvernoy/pseudomonas/100806_pao1.txt",sep="\t")
# file_name = "pseudo1"

scale = 65* 1.e-3 # Scale for E.Coli
# scale = 64.5* 1.e-3 # Scale for Pseudomonas

df = cleaning(df)

# Applying the scale
df["x"] = df["x"]*scale
df["y"] = df["y"]*scale
df["length"] = df["length"]*scale
df["width"] = df["width"]*scale

df.loc[df["time"]==df["time"].max()]

,cname,frame,time,cellno,schnitzno,birth,lifespan,length,area,angle,grate,width,x,y
3656,0HHTHHHTH,70,204,1,474,201,9,6.937190,1774,-1.682720,0.026161,1.440589,35.313200,76.428300
3657,0HTHHHTTH,70,204,2,457,198,12,5.310897,1156,-1.722780,0.029215,1.247591,38.833535,76.475750
3658,0HHTHTHTT,70,204,3,462,198,12,8.209240,1774,-2.812800,0.034031,1.254884,44.887375,70.634850
3659,0TTHHTHTH,70,204,4,372,189,21,6.452076,1496,0.121166,0.024240,1.294820,56.130815,77.798500
3660,0HHHTTHTH,70,204,5,386,195,15,9.183980,1842,0.591116,0.029354,1.397337,58.844825,62.799425
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3988,0HHHHTHHH,70,204,342,532,204,6,6.469469,1565,-1.776550,0.000000,1.361945,68.244150,55.021265
3989,0TTHHTHTTT,70,204,343,661,207,3,4.093687,885,0.539489,0.000000,1.229065,60.616205,78.746200
3990,0HTHHTHHT,70,204,344,416,192,18,6.947265,1612,-2.794140,0.032883,1.368198,47.687640,72.866950
3991,0HTHHTTHH,70,204,345,494,201,9,5.538890,1179,0.586614,0.020692,1.205282,63.385010,75.171850


## File preparation

In [61]:
# Opening the final file
file = open("./Data_Duvernoy_converted/Coli/"+ file_name + ".txt","a")
# file = open("./Data_Duvernoy_converted/pseudomonas/"+ file_name + ".txt","a")

# Writing the columns names
file.write("N disks\t")
file.write("Disks radius\t")
file.write("time\t")
file.write("Gen\t")
file.write("ks\t")
file.write("kt_bot\t")
file.write("kt_par\t")
file.write("kc\t")
file.write("color\t")
file.write("current length\t")
file.write("div_length\t")
file.write("X")
file.write("\n")


1

## Writing the new data

In [62]:

for i,row in df.iterrows():
    # Writing the number of disks 
    file.write(f"{3}\t")

    # Writing the radius
    radius = row["width"]/2
    file.write(f"{radius}\t")

    # Writing the time
    temp = row["time"]
    file.write(f"{temp:.3f}\t")

    # Writing the name
    temp = row["cname"]
    file.write(f"{len(temp)}\t")

    # Writing the stifnesses
    file.write(f"{0}\t")
    file.write(f"{0}\t")
    file.write(f"{0}\t")
    file.write(f"{0}\t")
    file.write(f"{(0,0,0)}\t")

    # Writing the length
    temp = row["length"]
    file.write(f"{temp}\t")
    file.write(f"{temp}\t")

    # writing the disks positions
    xc = row["x"]
    yc = row["y"]

    angle = row["angle"]
    file.write(f"{xc - (temp/2-radius)*np.cos(angle)} ")
    file.write(f"{yc - (temp/2-radius)*np.sin(angle)} ")

    file.write(f"{xc} ")
    file.write(f"{yc} ")

    file.write(f"{xc + (temp/2-radius)*np.cos(angle)} ")
    file.write(f"{yc + (temp/2-radius)*np.sin(angle)} ")
    file.write("\n")

file.close()